In [1]:
import time
import numpy as np
import pandas as pd
import random
import copy, sys
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from sklearn import ensemble
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing

In [2]:
#Function to load the data file to memeory.

#Input: File path to read.
#Output: A 2d numpy array with all loaded samples from the file to read in string.

def parseFile_raw(file):
    time_start = time.time()

    content = []
    count, count_incomplete,count_complete, count_part = 0, 0, 0, 0
    
    with open(file) as txtfile:
        for row in txtfile:
            
            row = row.split(',')
            row[-1] = row[-1].strip()
            #if count != 0:
            content.append([row[21]] + row[0:4] + [row[22]] + [row[32]] + row[24: 26] + [row[29]] + [row[6]] \
                           + [row[-5]] + [row[-4]] + [row[-3]] + [row[-2]] + [row[12].strip("'")])

            count += 1
            #if count == 1000:
                #break

    content_mat = np.array(content)

    time_end = time.time()
    print('Reading data is complete! Running time is ' + str(time_end - time_start) + 's!')

    return content_mat

In [3]:
#Function to load the data file to memeory, this is for the simulation hash data.

#Input: File path to read.
#Output: A 2d numpy array with all loaded samples from the file to read in string.

def parseFile_reference(file):
    time_start = time.time()

    content = []
    count, count_incomplete,count_complete, count_part = 0, 0, 0, 0
    
    with open(file) as txtfile:
        for row in txtfile:
            row = row.split(',')
            row[-1] = row[-1].strip().strip(']').strip('\n')
            row[0] = row[0][1:]
            row[0] = row[0].strip("'")
    
            content.append(row)

    reference_mat = np.array(content)

    time_end = time.time()
    print('Reading data is complete! Running time is ' + str(time_end - time_start) + 's!')

    return reference_mat

In [4]:
def parseFile_indi(file):

    with open(file, 'r') as csvfile:
        indi_list = []
        for line in csvfile:
            indi_list.append(line.strip().replace('-', ' ').split(','))

    indicator_array = np.array(indi_list)
    
    return indicator_array

def parseFile_hpi(file):

    with open(file, 'r') as csvfile:
        hpi_list = []
        for line in csvfile:
            hpi_list.append(line.strip().split(','))

    hpi_array = np.array(hpi_list)
    
    return hpi_array

In [5]:
#Function to filter the samples with no missing values. 
#Input: mat - 2d Numpy Array.
#Onput: mat - 2d Numpy Array with all samples that have no Missing values.

def filter_full_feature(mat):
    row_count = 0
    full_list = []
    for row in mat:
        if 'N/A' in row or 'NA' in row:
            pass
        else:
            full_list.append(row_count)

        row_count += 1
    print('There are a total of ' + str(len(full_list)) + ' samples fed into the model')
    mat = mat[full_list, :]
    return mat

#Function to split the fullset into training and test sets.
#Input: mat - 2d Numpy Array.
#Onput: train_mat: 2d Numpy Array, test_mat: 2d Numpy Array
def train_test_split(mat):
    train_list = []
    test_list = []
    num_sample, num_var = mat.shape

    for i in range(0, num_sample):
        if i == 0:
            train_list.append(i)
            test_list.append(i)
        else:
            rand = random.random()
            if rand >= 0.1:
                train_list.append(i)
            else:
                test_list.append(i)

    train_mat = mat[train_list, :]
    test_mat = mat[test_list, :]

    return train_mat, test_mat

In [6]:
#convert a probability into the coordinate of a zip code using population probability distritbuion
#prob: float between 0 and 1
#reference_array: a 2-d array contaning the coordinates of the reference zipcodes
#prob_dist: a 1-d array shows the accumulated population distribution as a percentage of the total population in the US.
def getzip(prob, reference_array, prob_dist):
    idx = np.where(prob_dist >= prob)
    idx = idx[0][0]
    coord = reference_array[idx, :]
    
    return coord, idx

#convert the index and probability from getzip() and get the gender of the simulation from a gender reference
#idx: the index returned from getzip()
#gender_ref: a 2-d Array that contains the gender distribution of each zip code.
# 1-male, 0-female.
def getgender(idx, gender_ref):
    prob = random.random()
    
    if prob >= gender_ref[idx]:
        gender = 0
    else:
        gender = 1
    
    return gender

#convert the index, gender and a probability from getzip() and get the age of the simulation from an age reference
def getage(idx, age_ref, gender):
    age_ref_male = age_ref[:, :18]
    age_ref_female = age_ref[:, 18:]
    prob = random.random()
    
    if gender == 1:
        idx_age = np.where(age_ref_male[idx] >= prob)
        if idx_age[0].size != 0:
            idx_age = idx_age[0][0]
            delta = random.randint(0, 4)
        
            age = idx_age * 5 + delta
        else:
            age = 90
    else:
        idx_age = np.where(age_ref_female[idx] >= prob)
        if idx_age[0].size != 0:
            idx_age = idx_age[0][0]
            delta = random.randint(0, 4)
        
            age = idx_age * 5 + delta
        else:
            age = 90

    return age

#convert the index and a probability from getzip() and get the race of the simulation from a race reference
def getrace(idx, race_ref):
    prob = random.random()
    idx_race = np.where(race_ref[idx] >= prob)
    idx_race = idx_race[0][0]
    
    return idx_race + 1


In [7]:
#convert a zip code to its coresponding coordinate.
#zip_array: a 1-d array that is a list of zip_code
#reference_array: a 2-d array contaning the coordinates of the reference zipcodes
def zip_to_coordinate(zip_array, reference_array):
    count = 0
    coordinate_list = []
    full_list = []
    zip_ref = reference_array[:, 0].astype(np.int)
    for zip_c in zip_array:
        idx = np.argwhere(zip_ref == int(zip_c))
        if idx.size != 0:
            coordinate_pair = reference_array[idx[0][0], 1:3]
            full_list.append(count)
        else: #there are some zipcodes were P.O box addresses and not in our reference. So we look for the nearby zipcodes
            zip_c_back = int(zip_c) - 1
            zip_c_forward = int(zip_c) + 1
            idx_back = np.argwhere(zip_ref == zip_c_back)
            idx_forward = np.argwhere(zip_ref == zip_c_forward)
            if idx_back.size != 0:
                coordinate_pair = reference_array[idx_back[0][0], 1:3]
                full_list.append(count)
            elif idx_forward.size != 0:
                coordinate_pair = reference_array[idx_forward[0][0], 1:3]
                full_list.append(count)
            else:
                coordinate_pair = ['N/A', 'N/A']
                
        count += 1
        coordinate_list.append(coordinate_pair)
    return np.array(coordinate_list), full_list

In [8]:
def random_shuffle(array, upper_array, lower_array):
    element_counter = 0
    for element in array[0]:
        prob = random.random()
        if prob <= 0.15:
            array[0][element_counter] = np.random.choice(np.arange(lower_array[element_counter], upper_array[element_counter]))  
        element_counter += 1
        
    return array

def convert_dummy(array):
    
    num_sample, num_feature = array.shape
    dummy_list = []
    
    combined_df = np.array(pd.get_dummies(array[:, 0]))
    for i in range(1, num_feature):
        dummy_df = pd.get_dummies(array[:, i])
        combined_df = np.concatenate((combined_df, np.array(dummy_df)), axis=1)
    
    return combined_df

In [9]:
def simulation(neigh_model, zip_array ,coordinate_array, gender_array, age_array, race_array, prob_dist, daily_indicator, year, norm_model):
    
    # generate a random probability prop to population distri. (use zip for now)
    prob = random.random() # 0.0~1.0
    
    #longi, lati = getcoord(zip)
    coordinate, idx = getzip(prob, coordinate_array, prob_dist)
    
    #zip_c = int(zip_array[idx])
    
    gender = getgender(idx, gender_array)
    
    x_knn = np.append(coordinate, gender)
    
    age = getage(idx, age_array, gender)
    
    x_knn = np.append(x_knn, age)
    
    race = getrace(idx, race_array)
    
    x_knn = np.append(x_knn, race)
    
    x_knn = np.append(x_knn, daily_indicator)
    
    '''
    
    hpi_array = parseFile_hpi("CleanedData/hpi_cleaned.csv")
    hpi_locator = hpi_array[1:, 0:2].astype(np.int)
    idx_hpi = np.argwhere(np.logical_and(hpi_locator[:,0] == zip_c, hpi_locator[:,1] == year))
    
    if idx_hpi.size != 0:
        x_knn = np.append(x_knn, float(hpi_array[idx_hpi[0][0], 2])).reshape(1, 9)
    else:
        x_knn = np.append(x_knn, 0).reshape(1, 9)
    '''
    x_knn_for_tran = copy.deepcopy(x_knn)
    scaled_x_knn = norm_model.transform([x_knn_for_tran])
    
    
    # generate sentiment features (use knn for now)
    senti_feature = neigh_model.predict(scaled_x_knn.reshape(1,8))
    
    upper_limit = [5, 2, 11, 11, 2, 4, 2, 2]
    lower_limit = [1, -1, 0, 0, 0, 1, 0, 0]
    
    senti_feature = random_shuffle(senti_feature, upper_limit, lower_limit)
    
    
    return senti_feature, x_knn

In [10]:
def model_train(mat, label_location):
    #model = linear_model.LogisticRegression()
    poly = preprocessing.PolynomialFeatures(2)
    num_sam, num_var = mat.shape
    model = ensemble.RandomForestClassifier(n_estimators = 15,min_samples_split= 32, min_samples_leaf = 20)
    feature_mat = np.delete(mat, label_location, axis=1).astype(np.float)
    feature_mat = poly.fit_transform(feature_mat)
    #feature_mat = np.concatenate((feature_mat, (feature_mat[:, 41] * feature_mat[:, 41]).reshape((num_sam, 1))), axis=1)
    labels = mat[:, label_location].astype(np.int)
    print('Model training - Started!')
    time_start = time.time()
    model.fit(feature_mat, labels)
    time_end = time.time()
    print('Model training - Completed! Training time: ' + str(time_end - time_start) + 's')

    predicted_lab = model.predict(feature_mat)
    corrected_pred = np.sum(labels == predicted_lab)

    training_error = 1 - corrected_pred/labels.size

    return model, training_error


def model_test(model, mat, label_location):
    num_sam, num_var = mat.shape
    poly = preprocessing.PolynomialFeatures(2)
    feature_mat = np.delete(mat, label_location, axis=1).astype(np.float)
    feature_mat = poly.fit_transform(feature_mat)
    #feature_mat = np.concatenate((feature_mat, (feature_mat[:, 41] * feature_mat[:, 41]).reshape((num_sam, 1))), axis=1)
    labels = mat[:, label_location].astype(np.int)

    predicted_lab = model.predict(feature_mat)
    corrected_pred = np.sum(labels == predicted_lab)
    
    label_score = model.predict_proba(feature_mat)
    
    print('The current model stands an AUC of ' + str(roc_auc_score(labels, label_score[:, 1])))
    
    np.savetxt('predicted_lab_RF.txt', predicted_lab.astype(np.int))
    np.savetxt('label_test_RF.txt', labels.astype(np.int))

    test_error = 1 - corrected_pred / labels.size
    return test_error

def model_sim(model, mat):
    poly = preprocessing.PolynomialFeatures(2)
    feature_mat = mat.astype(np.float)
    feature_mat = poly.fit_transform(feature_mat)
    num_sim, num_var = feature_mat.shape
    #feature_mat = np.concatenate((feature_mat, (feature_mat[:, 41] * feature_mat[:, 41]).reshape((num_sim, 1))), axis=1)
    predicted_lab = model.predict(feature_mat).reshape(num_sim, 1)
    
    full_mat = np.concatenate((feature_mat, predicted_lab), axis=1)
    
    return full_mat

In [11]:
def main():
    file = "CleanedData/gallup_clean_NA_determinant.txt"
    file_age = "CleanedData/ppl_by_zip.txt"
    file_race = "CleanedData/race_by_zip.txt"
    
    file_indi = "CleanedData/daily_ind_1.csv"
    file_indi_oil = "CleanedData/daily_ind_oil.csv"
    file_indi_S = "CleanedData/daily_ind_SP500.csv"
    
    simu_iter = 50000        #327500000 is current US population
    
    raw_data = parseFile_raw(file) #raw_data from Gallup daily survey
    header = raw_data[0,:]
    cleaned_data_input = filter_full_feature(raw_data)[1:,:]  #cleaned_data input from Gallup
    
    label = cleaned_data_input[:, :1] #employmed label
    cleaned_data = cleaned_data_input[:, 1:]
    
    age_data = parseFile_reference(file_age)
    age_data = filter_full_feature(age_data)[1:,:]
    coordinate = age_data[:,2:4].astype(np.float) # (longi,lati)
    index = age_data[:,-1].astype(np.float) # prob
    age_dist = age_data[:, 5:-2].astype(np.float)
    
    race_data = parseFile_reference(file_race)
    race_data = filter_full_feature(race_data)[1:,:]
    race_dist = race_data[:, 2:].astype(np.float)
    
    zipcode = age_data[:,:1]
    gender_distribution = age_data[:, 4:5].astype(np.float)
    
    combined_zip_ref = np.concatenate((zipcode, coordinate), axis=1)
    
    zip_code_ind = cleaned_data[:, -1] 
    ind_coordinate, full_list = zip_to_coordinate(zip_code_ind, combined_zip_ref) # coreponding coordinate of the samples.
    
    content_mat = np.concatenate((cleaned_data,ind_coordinate),axis=1)
    content_mat = content_mat[full_list, :]
    label = label[full_list, :]
    
    num_sam, num_var = content_mat.shape
    
    X_knn = content_mat[:, -2:num_var + 1].astype(np.float)# zip vec
    gender = content_mat[:, 8].astype(np.int)
    age = content_mat[:, 9].astype(np.int)
    race = content_mat[:, 10].astype(np.int)
    daily_ind =  content_mat[:, 11:14].astype(np.float)
    #hpi = content_mat[:, -4].astype(np.float)
    #X_knn = np.concatenate((X_knn, gender.reshape((num_sam, 1)), age.reshape((num_sam, 1)), race.reshape((num_sam, 1)),daily_ind.reshape((num_sam, 3)), hpi.reshape(num_sam, 1) ),axis=1)
    X_knn = np.concatenate((X_knn, gender.reshape((num_sam, 1)), age.reshape((num_sam, 1)), race.reshape((num_sam, 1)),daily_ind.reshape((num_sam, 3))),axis=1)
    y_knn = content_mat[:,:8].astype(np.int) # senti mat
    
    
    X_for_transform = copy.deepcopy(X_knn)
    
    time_s = time.time()
    print('normalization starts!')
    scaler = preprocessing.MinMaxScaler()
    scaler.fit(X_knn)
    scaled_X_knn = scaler.transform(X_for_transform)
    time_s_end = time.time()
    print('normalization ends after ' + str(time_s_end - time_s))
    
    scaled_X_knn = np.around(scaled_X_knn, decimals = 10)
    
    sim_prob_ref = age_data[:, -1].astype(np.float)
    
    print('KNN starts!')
    neigh = KNeighborsClassifier(n_neighbors=15, weights= 'distance')
    neigh.fit(scaled_X_knn, y_knn)
    print('KNN ends!')
    
    dummy_y = convert_dummy(y_knn)
    
    content_mat = np.concatenate((label, dummy_y, X_knn), axis=1)
    train_mat, test_mat = train_test_split(content_mat)
    
    model, train_error = model_train(train_mat, 0)
    test_error = model_test(model, test_mat, 0)
    
    print('The training error for this trail is: ' + str(train_error))
    print('The testing error for this trail is: ' + str(test_error))
    
    indi_array = parseFile_indi(file_indi)
    year_list = indi_array[:, 0]
    indi_list = indi_array[:, 1].astype(np.float)
    oil_array = parseFile_indi(file_indi_oil)
    ind_oil_list = oil_array[:,1].astype(np.float)
    S_array = parseFile_indi(file_indi_S)
    ind_S_list = S_array[:,1].astype(np.float)
    
    num_sim_r = indi_list.shape
    
    employment_rate_16 = []
    employment_rate_25_54 = []
    
    #employment_rate_18.append('>18')
    #employment_rate_25_54.append('25 - 54')
    count = 0
    for index in range(0, num_sim_r[0]):
        time_start = time.time()
        print('Started simulation cycle ' + str(count))
        
        ind_list = [indi_list[index], ind_oil_list[index], ind_S_list[index]]
        year = int(year_list[index][-2:])
        X_classify = []
        coord_list = []
        for i in range(simu_iter):
            #senti_fea, coord = simulation(neigh, scaler, coordinate, gender_distribution, age_dist, race_dist, sim_prob_ref, ind_list)
            senti_fea, coord = simulation(neigh, zipcode, coordinate, gender_distribution, age_dist, race_dist, sim_prob_ref, ind_list, year, scaler)
            coord_list.append(coord)
            X_classify.append(senti_fea[0])
          
    # throw simulated data into the model, predict their unemplotment rate
        X_classify = np.array(X_classify)
        coord_list = np.array(coord_list)
        
        dummy_classified = convert_dummy(X_classify)
        
        output_array = np.concatenate((dummy_classified, coord_list), axis=1)
        
        sim_result = model_sim(model, output_array)
        
        age_array = output_array[:, 41].reshape((simu_iter, 1))
        employment_array = sim_result[:, -1].reshape((simu_iter, 1))
        
        over_16_list = np.argwhere(age_array >= 16)
        age_25_54_list = np.argwhere((age_array >= 25) & (age_array <= 54))
        #over_18_population = over_18_list.shape
        num_over_16, dim = over_16_list.shape
        num_25_54, dim = age_25_54_list.shape
        
        employment_array_16 = employment_array[over_16_list[:,0]]
        employment_array_25_54 = employment_array[age_25_54_list[:,0]]
        
        employment_rate_16.append(np.sum(employment_array_16)/num_over_16)
        employment_rate_25_54.append(np.sum(employment_array_25_54)/num_25_54)
        
        time_end = time.time()
        print('Simulation Cycle ' + str(count) + ' finished in ' + str(time_end - time_start) + 's!')
        count += 1
        
        #if count == 10:
            #break
        
    age_16 = np.array(employment_rate_16).reshape((count, 1))
    age_25_54 = np.array(employment_rate_25_54).reshape((count, 1))
    
    #ind_list_out = ind_list.reshape(num_sim, 1)
    
    output_array = np.concatenate((age_16, age_25_54), axis=1)
    np.savetxt('sim_out_daily_RF_Norm'+ str(simu_iter) + '.txt', output_array, delimiter=',', fmt='%1.4f,%1.4f')
    

if __name__ == "__main__":    
    main()

Reading data is complete! Running time is 31.552173614501953s!
There are a total of 857696 samples fed into the model
Reading data is complete! Running time is 0.7691481113433838s!
There are a total of 32800 samples fed into the model
Reading data is complete! Running time is 0.20369482040405273s!
There are a total of 32800 samples fed into the model
normalization starts!
normalization ends after 0.21509075164794922
KNN starts!
KNN ends!
Model training - Started!
Model training - Completed! Training time: 190.0780689716339s
The current model stands an AUC of 0.8173072330911594
The training error for this trail is: 0.2123007300671682
The testing error for this trail is: 0.24775935012412442
Started simulation cycle 0
Simulation Cycle 0 finished in 90.04291534423828s!
Started simulation cycle 1
Simulation Cycle 1 finished in 90.41172099113464s!
Started simulation cycle 2
Simulation Cycle 2 finished in 89.87560606002808s!
Started simulation cycle 3
Simulation Cycle 3 finished in 89.3637056

Simulation Cycle 94 finished in 95.69293308258057s!
Started simulation cycle 95
Simulation Cycle 95 finished in 93.82640552520752s!
Started simulation cycle 96
Simulation Cycle 96 finished in 92.47449731826782s!
Started simulation cycle 97
Simulation Cycle 97 finished in 91.92478585243225s!
Started simulation cycle 98
Simulation Cycle 98 finished in 96.06347727775574s!
Started simulation cycle 99
Simulation Cycle 99 finished in 95.96028017997742s!
Started simulation cycle 100
Simulation Cycle 100 finished in 97.50289845466614s!
Started simulation cycle 101
Simulation Cycle 101 finished in 97.0474181175232s!
Started simulation cycle 102
Simulation Cycle 102 finished in 96.3693118095398s!
Started simulation cycle 103
Simulation Cycle 103 finished in 99.99660682678223s!
Started simulation cycle 104
Simulation Cycle 104 finished in 97.86767673492432s!
Started simulation cycle 105
Simulation Cycle 105 finished in 94.67701935768127s!
Started simulation cycle 106
Simulation Cycle 106 finished

Simulation Cycle 194 finished in 94.84993267059326s!
Started simulation cycle 195
Simulation Cycle 195 finished in 96.06062602996826s!
Started simulation cycle 196
Simulation Cycle 196 finished in 95.34446597099304s!
Started simulation cycle 197
Simulation Cycle 197 finished in 93.15768933296204s!
Started simulation cycle 198
Simulation Cycle 198 finished in 91.75428485870361s!
Started simulation cycle 199
Simulation Cycle 199 finished in 93.07897901535034s!
Started simulation cycle 200
Simulation Cycle 200 finished in 95.90227627754211s!
Started simulation cycle 201
Simulation Cycle 201 finished in 98.975656747818s!
Started simulation cycle 202
Simulation Cycle 202 finished in 100.57257199287415s!
Started simulation cycle 203
Simulation Cycle 203 finished in 108.44458651542664s!
Started simulation cycle 204
Simulation Cycle 204 finished in 94.66929578781128s!
Started simulation cycle 205
Simulation Cycle 205 finished in 95.94325113296509s!
Started simulation cycle 206
Simulation Cycle

Simulation Cycle 294 finished in 86.01024961471558s!
Started simulation cycle 295
Simulation Cycle 295 finished in 86.89653992652893s!
Started simulation cycle 296
Simulation Cycle 296 finished in 87.67859625816345s!
Started simulation cycle 297
Simulation Cycle 297 finished in 87.30592966079712s!
Started simulation cycle 298
Simulation Cycle 298 finished in 86.3353168964386s!
Started simulation cycle 299
Simulation Cycle 299 finished in 86.43511414527893s!
Started simulation cycle 300
Simulation Cycle 300 finished in 88.62837314605713s!
Started simulation cycle 301
Simulation Cycle 301 finished in 87.30772233009338s!
Started simulation cycle 302
Simulation Cycle 302 finished in 88.74511003494263s!
Started simulation cycle 303
Simulation Cycle 303 finished in 91.41976976394653s!
Started simulation cycle 304
Simulation Cycle 304 finished in 87.51136565208435s!
Started simulation cycle 305
Simulation Cycle 305 finished in 87.48584461212158s!
Started simulation cycle 306
Simulation Cycle 

Simulation Cycle 404 finished in 90.9786229133606s!
Started simulation cycle 405
Simulation Cycle 405 finished in 93.2135317325592s!
Started simulation cycle 406
Simulation Cycle 406 finished in 92.33632326126099s!
Started simulation cycle 407
Simulation Cycle 407 finished in 90.64416074752808s!
Started simulation cycle 408
Simulation Cycle 408 finished in 90.24801063537598s!
Started simulation cycle 409
Simulation Cycle 409 finished in 90.77932167053223s!
Started simulation cycle 410
Simulation Cycle 410 finished in 90.52141857147217s!
Started simulation cycle 411
Simulation Cycle 411 finished in 92.88550281524658s!
Started simulation cycle 412
Simulation Cycle 412 finished in 91.33794784545898s!
Started simulation cycle 413
Simulation Cycle 413 finished in 93.61713099479675s!
Started simulation cycle 414
Simulation Cycle 414 finished in 91.80559515953064s!
Started simulation cycle 415
Simulation Cycle 415 finished in 91.74381160736084s!
Started simulation cycle 416
Simulation Cycle 4

...

Simulation Cycle 2286 finished in 96.53166103363037s!
Started simulation cycle 2287
Simulation Cycle 2287 finished in 95.43040728569031s!
Started simulation cycle 2288
Simulation Cycle 2288 finished in 89.89150524139404s!
Started simulation cycle 2289
Simulation Cycle 2289 finished in 89.69264531135559s!
Started simulation cycle 2290
Simulation Cycle 2290 finished in 98.82026195526123s!
Started simulation cycle 2291
Simulation Cycle 2291 finished in 104.4005720615387s!
Started simulation cycle 2292
Simulation Cycle 2292 finished in 108.94960284233093s!
Started simulation cycle 2293
Simulation Cycle 2293 finished in 107.09893584251404s!
Started simulation cycle 2294
Simulation Cycle 2294 finished in 103.77057409286499s!
Started simulation cycle 2295
Simulation Cycle 2295 finished in 105.61753726005554s!
Started simulation cycle 2296
Simulation Cycle 2296 finished in 102.76547360420227s!
Started simulation cycle 2297
Simulation Cycle 2297 finished in 102.33580112457275s!
Started simulati

Simulation Cycle 2384 finished in 96.2908341884613s!
Started simulation cycle 2385
Simulation Cycle 2385 finished in 94.27813744544983s!
Started simulation cycle 2386
Simulation Cycle 2386 finished in 88.04559373855591s!
Started simulation cycle 2387
Simulation Cycle 2387 finished in 93.0251247882843s!
Started simulation cycle 2388
Simulation Cycle 2388 finished in 99.84241366386414s!
Started simulation cycle 2389
Simulation Cycle 2389 finished in 98.95251274108887s!
Started simulation cycle 2390
Simulation Cycle 2390 finished in 98.93025755882263s!
Started simulation cycle 2391
Simulation Cycle 2391 finished in 98.12258982658386s!
Started simulation cycle 2392
Simulation Cycle 2392 finished in 95.25218987464905s!
Started simulation cycle 2393
Simulation Cycle 2393 finished in 97.12538862228394s!
Started simulation cycle 2394
Simulation Cycle 2394 finished in 100.55720090866089s!
Started simulation cycle 2395
Simulation Cycle 2395 finished in 97.33377242088318s!
Started simulation cycl

Simulation Cycle 2482 finished in 88.44159150123596s!
Started simulation cycle 2483
Simulation Cycle 2483 finished in 88.43392968177795s!
Started simulation cycle 2484
Simulation Cycle 2484 finished in 93.53609347343445s!
Started simulation cycle 2485
Simulation Cycle 2485 finished in 89.92590188980103s!
Started simulation cycle 2486
Simulation Cycle 2486 finished in 86.35851836204529s!
Started simulation cycle 2487
Simulation Cycle 2487 finished in 94.07896423339844s!
Started simulation cycle 2488
Simulation Cycle 2488 finished in 94.12855744361877s!
Started simulation cycle 2489
Simulation Cycle 2489 finished in 95.53199648857117s!
Started simulation cycle 2490
Simulation Cycle 2490 finished in 94.61202549934387s!
Started simulation cycle 2491
Simulation Cycle 2491 finished in 92.13556718826294s!
Started simulation cycle 2492
Simulation Cycle 2492 finished in 91.56956577301025s!
Started simulation cycle 2493
Simulation Cycle 2493 finished in 95.00691866874695s!
Started simulation cyc

Simulation Cycle 2580 finished in 92.94052052497864s!
Started simulation cycle 2581
Simulation Cycle 2581 finished in 92.08895492553711s!
Started simulation cycle 2582
Simulation Cycle 2582 finished in 94.60694980621338s!
Started simulation cycle 2583
Simulation Cycle 2583 finished in 94.5922520160675s!
Started simulation cycle 2584
Simulation Cycle 2584 finished in 93.46298718452454s!
Started simulation cycle 2585
Simulation Cycle 2585 finished in 96.94212889671326s!
Started simulation cycle 2586
Simulation Cycle 2586 finished in 92.40179681777954s!
Started simulation cycle 2587
Simulation Cycle 2587 finished in 86.97892093658447s!
Started simulation cycle 2588
Simulation Cycle 2588 finished in 86.13759183883667s!
Started simulation cycle 2589
Simulation Cycle 2589 finished in 84.49009847640991s!
Started simulation cycle 2590
Simulation Cycle 2590 finished in 85.88375878334045s!
Started simulation cycle 2591
Simulation Cycle 2591 finished in 85.46266269683838s!
Started simulation cycl

Simulation Cycle 2678 finished in 105.78744602203369s!
Started simulation cycle 2679
Simulation Cycle 2679 finished in 112.0520408153534s!
Started simulation cycle 2680
Simulation Cycle 2680 finished in 107.28912281990051s!
Started simulation cycle 2681
Simulation Cycle 2681 finished in 106.61809086799622s!
Started simulation cycle 2682
Simulation Cycle 2682 finished in 101.40173578262329s!
Started simulation cycle 2683
Simulation Cycle 2683 finished in 99.05383920669556s!
Started simulation cycle 2684
Simulation Cycle 2684 finished in 97.08794808387756s!
Started simulation cycle 2685
Simulation Cycle 2685 finished in 97.70896863937378s!
Started simulation cycle 2686
Simulation Cycle 2686 finished in 98.59763169288635s!
Started simulation cycle 2687
Simulation Cycle 2687 finished in 100.7611997127533s!
Started simulation cycle 2688
Simulation Cycle 2688 finished in 95.22131609916687s!
Started simulation cycle 2689
Simulation Cycle 2689 finished in 90.62304878234863s!
Started simulation